In [1]:
!pip install torch transformers

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [4]:
import pandas as pd
#leemos el dataset
data =pd.read_csv('/content/drive/MyDrive/2-SIS421/ExamenFinal/medtext_2.csv')
data

,Prompt,Completion
0,A 50-year-old male presents with a history of ...,This patient's history of recurrent kidney sto...
1,"A 7-year-old boy presents with a fever, headac...","This child's symptoms of a red, bulging tympan..."
2,A 35-year-old woman presents with a persistent...,While the symptoms might initially suggest ast...
3,A 50-year-old male presents with severe abdomi...,The patient's symptoms suggest an incarcerated...
4,A newborn baby presents with eye redness and a...,The infant's symptoms suggest neonatal conjunc...
...,...,...
1407,A 55-year-old male with a history of chronic o...,While this patient's symptoms could be due to ...
1408,Can diet and lifestyle changes help manage vit...,While there is no specific diet or lifestyle m...
1409,A 50-year-old female presents with right shoul...,This patient's shoulder and arm pain following...
1410,A 60-year-old female with high cholesterol lev...,In addition to a diet low in saturated fats an...


In [5]:
import csv
import unicodedata #proporciona funciones para trabajar con caracteres Unicode.
import re  #ofrece operaciones de expresiones regulares
#Convierte una cadena Unicode s a su forma ASCII eliminando las marcas diacríticas
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s) #para normalizar la cadena al formato de descomposición canónica (NFD)
        if unicodedata.category(c) != 'Mn'  #filtra los caracteres que no son marcadores no espaciadores (Mn)
    )
#Normaliza una cadena s
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z0-9.!?]+", r" ", s)  # Conserva números
    return s


preguntas = []
respuestas = []

with open('/content/drive/MyDrive/2-SIS421/ExamenFinal/medtext_2.csv', 'r') as file:
    reader = csv.reader(file)
    next(reader)  # Ignorar la primera fila (encabezados)
    for row in reader:
        #normaliza las preguntas y respuestas utilizando las funciones definidas anteriormente
        pregunta = normalizeString(row[0])
        respuesta = normalizeString(row[1])
        preguntas.append(pregunta)
        respuestas.append(respuesta)

data = [{'question': q, 'answer': a} for q, a in zip(preguntas, respuestas)]

# Imprimir el primer diálogo
print(data[1])
print(len(data))
dialogues = data
print(len(dialogues))

{'question': 'a 7 year old boy presents with a fever headache and severe earache . he also complains of dizziness and a spinning sensation . examination reveals a red bulging tympanic membrane . what are the differential diagnoses and what should be done next ?', 'answer': 'this child s symptoms of a red bulging tympanic membrane with systemic symptoms such as fever and headache and the additional symptoms of dizziness and a spinning sensation raise concern for complications of acute otitis media . the differential diagnosis could include labyrinthitis or possibly even mastoiditis . urgent evaluation including further imaging studies such as a ct or mri scan may be necessary . this child likely requires admission for intravenous antibiotics and possibly surgical intervention if mastoiditis is confirmed .'}
1412
1412


In [6]:
max_words = 0
# Iterar sobre cada par de preguntas y respuestas en el conjunto de datos
for pair in dialogues:
    input_words = pair['question'].split()  # Dividir la entrada en palabras individuales
    output_words = pair['answer'].split()  # Dividir la salida en palabras individuales
    # Determinar el número máximo de palabras entre la pregunta y la respuesta
    num_words = max(len(input_words), len(output_words))

    if num_words > max_words:
        max_words = num_words

print("Máximo número de palabras:", max_words)


Máximo número de palabras: 120


In [7]:
import torch
#cargamos el tokenizador de blenderbot
from transformers import BlenderbotSmallTokenizer
tokenizer = BlenderbotSmallTokenizer.from_pretrained("facebook/blenderbot-90M")

vocab.json:   0%|          | 0.00/964k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/345k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

In [8]:
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, train_data, tokenizer, max_length):
        self.train_data = train_data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.train_data) # Devuelve la longitud total del conjunto de datos

    def __getitem__(self, idx):
        question = self.train_data[idx]['question'] # Obtiene la pregunta del par actual
        answer = self.train_data[idx]['answer'] # Obtiene la respuesta del par actual

        # Codificación de la pregunta
        question_encoding = self.tokenizer.encode_plus(
            question,
            add_special_tokens=True,
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )

        input_ids = question_encoding['input_ids'].squeeze() # ID de tokens de entrada para la pregunta
        attention_mask = question_encoding['attention_mask'].squeeze() # Máscara de atención para la pregunta

        # Codificación de la respuesta
        answer_encoding = self.tokenizer.encode_plus(
            answer,
            add_special_tokens=True,
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )

        target_ids = answer_encoding['input_ids'].squeeze()  # ID de tokens objetivo para la respuesta
        target_ids_mak = answer_encoding['attention_mask'].squeeze() # Máscara de atención para la respuesta

       # Devuelve un diccionario con los datos de para en par de las pregunta y respuesta
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'target_ids': target_ids,
            'attention_target': target_ids_mak
        }


In [9]:
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
#dividimos nuestros datos procesados  en entranamiento y prueba
train_size = len(dialogues) * 80 // 100
train_pairs = dialogues[:train_size]
test_pairs = dialogues[train_size:]

# Crear una instancia para el conjunto de datos  tokenizados
max_length = 130
train_dataset = CustomDataset(train_pairs, tokenizer, max_length)
test_dataset = CustomDataset(test_pairs, tokenizer, max_length)

# Creamos  DataLoader de entrenamiento y de test
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)


In [15]:
#Itereamos para ver si los resultados de los dataloaders estan correctamente
batch = next(iter(train_dataloader))
input_ids = batch['input_ids']
attention_inputs = batch['attention_mask']
target_ids = batch['target_ids']
attention_target = batch['attention_target']

#print(len(train_dataloader))
print(input_ids[0])
print(attention_inputs[0])
#print(target_ids.shape,target_ids[2])


tensor([   12,  1975,   169,   217,   386,    28,   317,  7748,  6645,    28,
        22816,  1584, 11225, 15864,    25,    33,  1174,     5,    34,  2884,
           22,   150, 15864,    49,    78,  1436,  5155,   101,     7,   703,
          487,   734,     5,    44,   107,    43,     7,   878,     9,   277,
         2369,    20,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0, 

In [16]:
# Obtén los datos relevantes del dataloader para guardarlos
import torch
train_data = {
    "pregunta_input_ids": input_ids,
    "attention_input": attention_inputs,
    "respuesta_ids": target_ids,
    "attention_respuesta": attention_target
}

# Guarda los datos en un archivo separado
torch.save(train_data, "train_data.pth")


In [17]:
# Cargar los datos desde el archivo
train_data = torch.load("train_data.pth")

# Acceder a los datos
pregunta_input_ids = train_data["pregunta_input_ids"]
attention_inputs = train_data["attention_input"]
respuesta_input_ids = train_data["respuesta_ids"]
attention_respuesta = train_data["attention_respuesta"]
print(attention_respuesta.shape)

torch.Size([16, 130])


In [18]:
# Imprimir las preguntas y respuestas del primer ejemplo en el lote
print("Pregunta:")
print(tokenizer.decode(pregunta_input_ids[0], skip_special_tokens=True))

print("Respuesta:")
print(tokenizer.decode(respuesta_input_ids[0], skip_special_tokens=True))

Pregunta:
a 35 year old woman with pcos presents with yellowish waxy bumps on her skin. she reports that these bumps have been slowly appearing over the past six months. what could be the cause and next steps?
Respuesta:
this patient s skin findings could be ive of eruptive xanthomas which are skin lesions caused by the deposition of lipids in the skin and are often associated with high levels of blood lipids. women with pcos are at higher risk for dyslipidemia due to insulin resistance. the next step would be to confirm this diagnosis with a lipid panel blood test. if her lipid levels are indeed high lifestyle modifications and potentially lipid lowering medications would be recommended.


In [19]:
import torch
#cargamos el Transformer blenderbot
from transformers import BlenderbotSmallForConditionalGeneration

model= BlenderbotSmallForConditionalGeneration.from_pretrained("facebook/blenderbot-90M")
#mostramos en modelo para ver su arquitectura
model

pytorch_model.bin:   0%|          | 0.00/350M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/311 [00:00<?, ?B/s]

BlenderbotSmallForConditionalGeneration(
  (model): BlenderbotSmallModel(
    (shared): Embedding(54944, 512, padding_idx=0)
    (encoder): BlenderbotSmallEncoder(
      (embed_tokens): Embedding(54944, 512, padding_idx=0)
      (embed_positions): BlenderbotSmallLearnedPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-7): 8 x BlenderbotSmallEncoderLayer(
          (self_attn): BlenderbotSmallAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512

Encoder (Codificador):

Toma una secuencia de entrada (por ejemplo, una oración en lenguaje natural).
Transforma cada elemento de la secuencia (por ejemplo, cada palabra en la oración) en un espacio vectorial (embedding).
Captura la información contextual y la representa en un conjunto de vectores.
Genera una representación contextualizada de la entrada, que puede contener información sobre la relación entre los elementos de la secuencia.

Decoder (Decodificador):

Toma la representación contextualizada generada por el encoder.
Genera una secuencia de salida paso a paso, prediciendo el siguiente elemento de la secuencia en cada paso.
Utiliza la información contextual del encoder y la información ya generada para predecir cada elemento siguiente de manera coherente.
La salida del decoder puede ser, por ejemplo, una traducción de la entrada (si estamos tratando con un modelo de traducción) o una respuesta generada (si estamos tratando con un modelo de chat).

In [20]:
import torch
import torch.nn as nn
class EncoderDecoder(torch.nn.Module):
    def __init__(self):
        super(EncoderDecoder, self).__init__()
        #cargamos el transformer
        self.blenderbot_model = BlenderbotSmallForConditionalGeneration.from_pretrained("facebook/blenderbot-90M")
        # Congelar todos los parámetros del modelo preentrenado
        for param in self.blenderbot_model.parameters():
            param.requires_grad = False
        # encoder del modelo
        self.encoder = self.blenderbot_model.model.encoder
        #decoder delo modelo
        self.decoder = self.blenderbot_model.model.decoder
        self.generate = self.blenderbot_model.model.decoder.layernorm_embedding
        #Es una capa lineal que se entrena para mapear la representación oculta de la última capa del decoder a la distribución de probabilidad sobre el vocabulario.
        self.lm_head = self.blenderbot_model.lm_head

        # Descongelar los parámetros de la última capa del encoder para fine-tuning
        for layer in self.encoder.layers[-1:]:
            for param in layer.parameters():
                param.requires_grad = True

        # Descongelar los parámetros de la última capa del decoder para fine-tuning
        for layer in self.decoder.layers[:-2]:
            for param in layer.parameters():
                param.requires_grad = False

        # Descongelar los parámetros de la última capa (lm_head) para fine-tuning
        #for param in self.lm_head.parameters():
         #   param.requires_grad = True


    def forward(self, input_ids, attention_mask, decoder_input_ids, decoder_attention_mask):
        with torch.no_grad():
            # introducir los valores que se nesecita para el encoder  y sacamos la ultima capa oculta del encoder
            encoder_outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state
            #print(encoder_outputs.shape)
            #introducir la salida del encoder  al decoder y tambien las las respuestas tokenizadas ,
            # sacamos la ultima capa del decoder
            decoder_outputs = self.decoder(
                    input_ids=decoder_input_ids,
                    attention_mask=decoder_attention_mask,
                    encoder_hidden_states=encoder_outputs
                ).last_hidden_state

            #sacamos el mapeo de la ultima capa del decoder
            lm_output = self.lm_head(decoder_outputs)

        return  lm_output


In [21]:
modelo = EncoderDecoder()
decoder_outputs = modelo(input_ids,attention_inputs,target_ids,attention_target)
print(decoder_outputs.shape,decoder_outputs[2])

torch.Size([16, 130, 54944]) tensor([[ -2.4774, -10.0391,   2.3303,  ...,  -2.4750,  -2.4827,  -2.4730],
        [ -3.7953,  -9.3873,   6.1254,  ...,  -3.7939,  -3.7974,  -3.7927],
        [ -3.2837,  -3.3455,   5.8868,  ...,  -3.2803,  -3.2869,  -3.2802],
        ...,
        [ -2.9225,  -6.7612,  10.0714,  ...,  -2.9196,  -2.9248,  -2.9206],
        [  2.2018,  -3.4307,   6.6625,  ...,   2.2059,   2.1962,   2.2098],
        [  2.2276,  -3.4458,   6.6977,  ...,   2.2316,   2.2220,   2.2356]])


In [22]:
from tqdm import tqdm
import numpy as np
import torch.nn.functional as F

def fit(model, dataloader,dataloader2 ,epochs=10):
    encoder_optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
    criterio=torch.nn.CrossEntropyLoss()

    for epoch in range(1, epochs+1):
        model.train()
        train_loss = []
        bar = tqdm(dataloader)
        for batch in bar:
            input_ids = batch['input_ids']
            attention_inputs = batch['attention_mask']
            target_ids = batch['target_ids']
            attention_target = batch['attention_target']

            encoder_optimizer.zero_grad()
            decoder_outputs = model(input_ids, attention_inputs, target_ids, attention_target)
            # Convertir el target en un tensor de probabilidades
            target_probs = F.one_hot(target_ids, num_classes=54944).float()  # ahora tiene forma (62, 130, 54944)
            target_ids=target_probs.requires_grad_()

            # Calcular la pérdida de entropía cruzada
            loss = F.binary_cross_entropy_with_logits(decoder_outputs, target_ids)

            loss.backward()
            encoder_optimizer.step()

            train_loss.append(loss.item())

        avg_loss = np.mean(train_loss)
        # Evaluar el modelo en el conjunto de datos de prueba
        model.eval()
        test_loss = []
        with torch.no_grad():
            for batch in test_dataloader:
                input_ids = batch['input_ids']
                attention_inputs = batch['attention_mask']
                target_ids = batch['target_ids']
                attention_target = batch['attention_target']

                decoder_outputs = model(input_ids, attention_inputs, target_ids, attention_target)
                target_probs = F.one_hot(target_ids, num_classes=54944).float()
                target_ids=target_probs.requires_grad_()
                loss = F.binary_cross_entropy_with_logits(decoder_outputs, target_ids)

                test_loss.append(loss.item())

        # Calcular la pérdida promedio de la evaluación
        avg_test_loss = np.mean(test_loss)

        # Imprimir los resultados de la época actual
        print('Epoch:', epoch, 'Train Loss:', avg_loss, 'Test Loss:', avg_test_loss)


In [23]:
fit(modelo,train_dataloader,test_dataloader,epochs=20)

100%|██████████| 71/71 [09:36<00:00,  8.12s/it]


Epoch: 1 Train Loss: 0.45514758246045717 Test Loss: 0.4208089874850379


100%|██████████| 71/71 [09:34<00:00,  8.09s/it]


Epoch: 2 Train Loss: 0.4479096636805736 Test Loss: 0.4201968212922414


100%|██████████| 71/71 [09:37<00:00,  8.14s/it]


...

Epoch: 18 Train Loss: 0.4459296532080207 Test Loss: 0.4213876525561015


100%|██████████| 71/71 [09:29<00:00,  8.03s/it]


Epoch: 19 Train Loss: 0.4419975217799066 Test Loss: 0.41862646407551235


100%|██████████| 71/71 [09:28<00:00,  8.01s/it]


Epoch: 20 Train Loss: 0.44539715370661775 Test Loss: 0.41825857758522034


In [24]:
import torch
import torch.nn as nn
checkpoint_path = "/content/drive/MyDrive/2-SIS421/ExamenFinal/checkpoint/modelo_entrenado1.pt"

# Guardar el modelo entrenado
torch.save(modelo.state_dict(), checkpoint_path)

In [25]:
import torch
import torch.nn as nn
checkpoint_path = "/content/drive/MyDrive/2-SIS421/ExamenFinal/checkpoint/modelo_entrenado1.pt"
# Cargar el modelo entrenado en el futuro
loaded_model = EncoderDecoder()  # Reemplaza "MyModel" con la clase de tu modelo
loaded_model.load_state_dict(torch.load(checkpoint_path))

<All keys matched successfully>

In [26]:
def predict(model, dataloader, tokenizer,input_ids,attention):
    model.eval()
    # Desactivar el cálculo de gradientes durante la predicción
    with torch.no_grad():

            # Generar la entrada para el decodificador
            decoder_input_ids = torch.ones_like(input_ids)
            decoder_attention_mask = attention

            # Pasar los datos al modelo
            with torch.no_grad():
              decoder_outputs = model(input_ids, attention, decoder_input_ids, decoder_attention_mask)
              # Aplicar muestreo multinomial a lm_logits
            temperature = 0.3  # Ajusta este valor según tus preferencias
            # al elejir una temperatura baja cerca de 0 tenderán a producir predicciones más deterministas y centradas en las opciones más probables,
            # mientras que si son mas altos los valores tienden a reproducir predicciones mas variables , resultados mas diversos

            # Aplicar muestreo estocástico con temperatura
            softmax_logits = torch.softmax(decoder_outputs / temperature, dim=-1)
            lm_probs_2d = softmax_logits.view(-1, softmax_logits.size(-1))
            sampled_ids = torch.multinomial(lm_probs_2d, num_samples=10)

            # Decodificar las predicciones a texto utilizando el tokenizador
            decoded_outputs = tokenizer.batch_decode(sampled_ids, skip_special_tokens=True)

    return decoded_outputs



In [27]:
# Tokenizar la pregunta y generar la máscara de atención
question="A 50-year-old male presents with a history of recurrent kidney stones and osteopenia. He has been taking high-dose vitamin D supplements due to a previous diagnosis of vitamin D deficiency. Laboratory results reveal hypercalcemia and hypercalciuria. What is the likely diagnosis, and what is the treatment?"
encoded_inputs = tokenizer(question, return_attention_mask=True,return_tensors="pt")
input_ids = encoded_inputs["input_ids"]
attention = encoded_inputs["attention_mask"]

In [29]:
salida = predict(loaded_model,test_dataloader,tokenizer,input_ids,attention)
print(salida[0])

i that he oh does is my well im wow


In [ ]:
#preguntas para pruebas
#A 45-year-old woman with fibromyalgia reports chronic insomnia. What strategies could be suggested to her to improve her sleep quality?,"Good sleep hygiene is often recommended as a first step in managing chronic insomnia associated with fibromyalgia. This includes maintaining a regular sleep schedule, making the sleep environment comfortable and free from distractions, avoiding caffeine and other stimulants before bed, and establishing a relaxing pre-sleep routine. In some cases, cognitive-behavioral therapy for insomnia may be recommended. If these strategies don't help, a healthcare provider may suggest a trial of medication."
#A 50-year-old male with Bipolar Disorder complains of dry mouth and constipation. His medications include lithium and olanzapine. What could be causing his symptoms?,"Both lithium and olanzapine, an atypical antipsychotic, can cause anticholinergic side effects, including dry mouth and constipation. Management includes advising the patient to drink ample fluids, maintain good oral hygiene, and use over-the-counter remedies for dry mouth. For constipation, increased dietary fiber, regular exercise, and potentially the use of laxatives are recommended."
#"A 40-year-old man presents with a firm, flesh-colored nodule with a central crater filled with keratin on his shoulder. It has grown slowly over the past few months. What could be the potential diagnosis?","The description of a flesh-colored, keratin-filled nodule suggests a diagnosis of keratoacanthoma. Keratoacanthoma is a low-grade skin tumor that resembles squamous cell carcinoma. While it often regresses spontaneously, it can be difficult to distinguish clinically from more serious skin cancers and therefore should be biopsied to confirm the diagnosis and to guide management."
#"A 55-year-old male with a history of chronic obstructive pulmonary disease presents with worsening shortness of breath and a productive cough with green sputum. He also has a low-grade fever and has lost weight unintentionally over the past couple of months. What are the potential causes, and what are the next steps?","While this patient's symptoms could be due to an exacerbation of his chronic obstructive pulmonary disease, the chronic nature of his symptoms, fever, and weight loss also raise concern for tuberculosis. Other differentials could include bronchiectasis, lung abscess, or malignancy. The next steps should include sputum samples for routine culture as well as acid-fast bacilli smear and culture, and a chest X-ray. Depending on the results, further imaging studies or bronchoscopy may be needed."
#Can diet and lifestyle changes help manage vitiligo?,"While there is no specific diet or lifestyle modification proven to prevent or cure vitiligo, maintaining a healthy lifestyle is always beneficial for overall health and well-being. Some people with vitiligo have reported that certain foods seem to exacerbate their condition, but this varies widely from person to person. It's also important to protect the skin from sunburn, as areas of vitiligo lack melanin and are more susceptible to sun damage."
#"A 50-year-old female presents with right shoulder and arm pain following a traffic accident where she was a passenger. She recalls her head being jolted backward and forward at the moment of the collision. She denies any weakness or numbness. What could be the possible cause, and how would you proceed?","This patient's shoulder and arm pain following a traffic accident where her head was jolted back and forth suggest a whiplash injury. The force exerted during the collision can cause strain to the muscles and ligaments of the neck, which can result in referred pain to the shoulder and arm. The lack of weakness or numbness is a good sign, indicating there may not be any nerve damage. The patient should be evaluated with a detailed physical examination, including a neurological evaluation. Pain management, including analgesics and muscle relaxants, might be needed, and physical therapy can be beneficial for recovery."
#A 60-year-old female with high cholesterol levels is interested in natural ways to manage her cholesterol. What might be a potential suggestion?,"In addition to a diet low in saturated fats and regular exercise, turmeric supplementation could be considered. Some studies suggest that curcumin may help lower LDL (""bad"") cholesterol and triglycerides, and increase HDL (""good"") cholesterol."
#What are the main subtypes of vitiligo and how do they differ in their presentation?,"The main subtypes of vitiligo are non-segmental vitiligo and segmental vitiligo. Non-segmental vitiligo, also known as generalized vitiligo, is the most common form and is characterized by depigmented patches that often appear on both sides of the body in a symmetrical pattern. Segmental vitiligo is less common and tends to occur at a younger age. It is characterized by one or a few depigmented patches that usually appear on one side of the body in a dermatomal pattern."


In [ ]:
# Guardar el punto de control al final de cada época
#       torch.save({
#            'epoch': epoch,
#            'model_state_dict': model.state_dict(),
 #           'optimizer_state_dict': encoder_optimizer.state_dict(),
  #          'loss': avg_loss,
   #     }, checkpoint_path)

In [ ]:
#checkpoint = torch.load("modelo_checkpoint.pt")
#model.load_state_dict(checkpoint['model_state_dict'])
#encoder_optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#start_epoch = checkpoint['epoch'] + 1
